# Air Stripping Model

In [ ]:
import watertap
from pyomo.environ import units as pyunits
from pyomo.util.check_units import assert_units_consistent
from idaes.core.util.model_statistics import degrees_of_freedom
from math import pi
import sys
import matplotlib.pyplot as plt
%matplotlib inline
import pyomo.util.infeasible as infeas
from watertap.core.util.infeasible import *
from idaes.core.solvers.get_solver import get_solver
from idaes.core.util.scaling import *
from pyomo.environ import *
set_scaling_factor
from idaes.core.util.model_statistics import number_variables, number_total_constraints

In [ ]:
Henrys_const = "calculated"
mass_transfer_coeff_kla = "calculated"
air_to_water_ratio = "calculated"

# Case Study 1
target_chems = ['1,1,1-Trichloroethane'] 
Ci =0.165  # mg/L
#Ce = 0.005  # mg/L
QgQl = 5
Ql = 0.158 # m3/s
Tl = 15 # C
Tg = 15 # C
Ionic_strength = 0.086 # M
dens_liq = 999.15 # kg/m3
dens_gas = 1.22 # kg/m3
visc_liq = 0.00115 # kg/m-s
visc_gas = 0.0000175 # kg/m-s

### Packing characteristics
pack_type = 'plastic'
at = 242
dp = 0.0889
Cf = 33
Stc = 0.0330
Pdrop = 75

"""
# Case Study 2
target_chems = ['o-Dichloropropane'] 
Ci = 1  # mg/L
#Ce = 0.1  # mg/L
QgQl = 60
Ql = 0.1 # m3/s
Tl = 10 # C
Tg = 10 # C
Ionic_strength = 0.012 # M
dens_liq = 999.7 # kg/m3
dens_gas = 1.247 # kg/m3
visc_liq = 0.001307 # kg/m-s
visc_gas = 0.0000179 # kg/m-s

### Packing characteristics
pack_type = 'plastic'
at = 125
dp = 0.0889
Cf = 39
Stc = 0.0330
Pdrop = 50
"""

# Dictionary of default chemical values
def get_chem_prop(chems):
    H_dim_data = {
        'Benzene': 0.216,
        'Ethylbenzene': 0.322,
        'Chlorobenzene': 0.147,
        'Propylbenzene': 0.442,
        'o-Dichlorobenzene': 0.064,
        'm-Dichlorobenzene': 0.117,
        'p-Dichlorobenzene': 0.130,
        'Toluene': 0.263,
        'Carbon Tetrachloride': 1.21,
        'o-Dichloropropane': 0.146,
        'o-Xylene': 0.199,
        'm-Xylene': 0.304,
        'p-Xylene': 0.304,
        'Chloroform': 0.172,
        '1,1,1-Trichloroethane': 0.712,
        'Methane': 28.4,
        'Carbon dioxide': 1.1,
        'Ammonia': 0.0006}
    
    MW_data = {
        'Benzene': 78.11,
        'Ethylbenzene': 106.17,
        'Chlorobenzene': 112.56,
        'Propylbenzene': 120.20,
        'o-Dichlorobenzene': 147,
        'm-Dichlorobenzene': 147,
        'p-Dichlorobenzene': 147,
        'Toluene': 92.14,
        'Carbon Tetrachloride': 153.82,
        'o-Dichloropropane': 112.98,
        'o-Xylene': 106.16,
        'm-Xylene': 106.16,
        'p-Xylene': 106.16,
        'Chloroform': 119.38,
        '1,1,1-Trichloroethane': 133.4,
        'Methane': 16.04,
        'Carbon dioxide': 44.01,
        'Ammonia': 17.03}

    Vc_data = {
        'Benzene': 256,
        'Ethylbenzene': 374,
        'Chlorobenzene': 303,
        'Propylbenzene': 441,
        'o-Dichlorobenzene': 360,
        'm-Dichlorobenzene': 359,
        'p-Dichlorobenzene': 364,
        'Toluene': 316,
        'Carbon Tetrachloride': 276,
        'o-Dichloropropane': 226,
        'o-Xylene': 369,
        'm-Xylene': 376,
        'p-Xylene': 379,
        'Chloroform': 239,
        '1,1,1-Trichloroethane': 294,
        'Methane': 98,
        'Carbon dioxide': 94,
        'Ammonia': 73}
    
    Enthalp_data = {
        'Benzene': 28.1,
        'Ethylbenzene': 39.4,
        'Chlorobenzene': 30.6,
        'Propylbenzene': 36.4,
        'o-Dichlorobenzene': 37.3,
        'm-Dichlorobenzene': 35.3,
        'p-Dichlorobenzene': 28.4,
        'Toluene': 32.4,
        'Carbon Tetrachloride': 33,
        'o-Dichloropropane': 31.1,
        'o-Xylene': 31.3,
        'm-Xylene': 38.6,
        'p-Xylene': 34.8,
        'Chloroform': 33.5,
        '1,1,1-Trichloroethane': 28.7,
        'Methane': 14.8,
        'Carbon dioxide': 19.9,
        'Ammonia': 36.1}

    log_Kow_data = {
        'Benzene': 2.13,
        'Ethylbenzene': 3.15,
        'Chlorobenzene': 2.84,
        'Propylbenzene': 3.69,
        'o-Dichlorobenzene': 3.38,
        'm-Dichlorobenzene': 3.53,
        'p-Dichlorobenzene': 3.44,
        'Toluene': 2.73,
        'Carbon Tetrachloride': 2.83,
        'o-Dichloropropane': 1.98,
        'o-Xylene': 3.12,
        'm-Xylene': 3.20,
        'p-Xylene': 3.15,
        'Chloroform': 1.97,
        '1,1,1-Trichloroethane': 2.49,
        'Methane': 1.09,
        'Carbon dioxide': 0.83,
        'Ammonia': 0.23}
    
    T_BoilingPoint_data = {
        'Benzene': 353.2,
        'Ethylbenzene': 409.1,
        'Chlorobenzene': 405.1,
        'Propylbenzene': 432.2,
        'o-Dichlorobenzene': 453.3,
        'm-Dichlorobenzene': 446.1,
        'p-Dichlorobenzene': 447.1,
        'Toluene': 383.7,
        'Carbon Tetrachloride': 349.9,
        'o-Dichloropropane': 369.1,
        'o-Xylene': 417.5,
        'm-Xylene': 412.1,
        'p-Xylene': 411.5,
        'Chloroform': 334.3,
        '1,1,1-Trichloroethane': 347.1,
        'Methane': 111.5,
        'Carbon dioxide': 194.7,
        'Ammonia': 239.8}
    
    chem_in = {
        'chem':[],
        'chem_H':{},
        'chem_MW':{},
        'chem_vc':{},
        'chem_enthalp':{},
        'chem_logKow':{},
        'chem_boiling':{},
        'chem_density':{},
        }
    
    for ch in chems:
        chem_in['chem'].append(ch)
        chem_in['chem_H'][ch] = H_dim_data[ch]
        chem_in['chem_MW'][ch] = MW_data[ch]
        chem_in['chem_vc'][ch] = Vc_data[ch]
        chem_in['chem_enthalp'][ch] = Enthalp_data[ch]
        chem_in['chem_logKow'][ch] = log_Kow_data[ch]
        chem_in['chem_boiling'][ch] = T_BoilingPoint_data[ch]

    return chem_in
chem_in = get_chem_prop(target_chems)
#print(chem_in)

if Henrys_const == "calculated":
    H = chem_in['chem_H'][target_chems[0]]
    kow = chem_in['chem_logKow'][target_chems[0]]
    enth = chem_in['chem_enthalp'][target_chems[0]]
else:
    H = float(input('What is the dimensionless Henry''s constant of the target pollutant? -->  '))

if mass_transfer_coeff_kla == "calculated":
    MW = chem_in['chem_MW'][target_chems[0]]
    vc = chem_in['chem_vc'][target_chems[0]]
    Tb = chem_in['chem_boiling'][target_chems[0]]
else:
    kLa = float(input('What is the overall mass transfer coefficient (Kla)? -->  '))

if air_to_water_ratio == "calculated":
    Ce = float(input('What is the target effluent? -->  '))

In [ ]:
m = ConcreteModel('--- Air Stripping Pre- and Post-treatment model')

# Parameters & Variables
#--------------------------------------------------------------------------------------------------------------------------
m.Ql = Param(initialize=Ql,
            units=pyunits.m**3 / pyunits.s,
            doc='Flowrate in m^3/s')

m.Tl = Param(initialize=Tl,
             within=NonNegativeReals,
             doc='Temperature of input liquid in °C')

m.Tg = Param(initialize=Tg,
             within=NonNegativeReals,
             doc='Temperature of input gas in °C')

m.pack_surface_area = Param(initialize=at,
              within=NonNegativeReals,
              units= 1 / pyunits.m,
              doc='Specific surface area of packing, 1/m')

m.pack_diam = Param(initialize=dp,
             within=NonNegativeReals,
             units=pyunits.m,
             doc='Nominal packing diameter, m')

m.pack_fact = Param(initialize=Cf,
             within=NonNegativeReals,
             units= 1 / pyunits.m,
             doc='Packing factor, 1/m')

m.Stc = Param(initialize=Stc,
              within=NonNegativeReals,
              doc='Critical surface tension of packing, kg/s^2')

m.Ci = Var(initialize=Ci,
             bounds=(1E-6, 1E5),
             units=pyunits.mg / pyunits.L,
             doc='Initial concentration of pollutant, mg/L')

m.Ce = Var(initialize=Ci*0.1,
           bounds=(1E-7, 1E3),
           units=pyunits.mg / pyunits.L,
           doc='Target exiting concentration of pollutant, mg/L')

m.Tl_K = Var(initialize=m.Tl+273.15,
             bounds=(0, 850),
             within=NonNegativeReals,
             units=pyunits.K,
             doc='Temperature of liquid in K')

m.Tg_K = Var(initialize=m.Tg+273.15,
             bounds=(0, 850),
             within=NonNegativeReals,
             units=pyunits.K,
             doc='Temperature of gas in K')

m.R = Var(initialize=0.008314,
          within=NonNegativeReals,
          units=pyunits.kJ / pyunits.mol * pyunits.K,
          doc='Ideal gas constant, kJ/mol-K')

m.g = Var(initialize=9.81,
          within=NonNegativeReals,
          units=pyunits.g / pyunits.s**2,
          doc='Gravitational constant, m/s^2')

m.dens_liq = Var(initialize=dens_liq,
          within=NonNegativeReals,
          units=pyunits.kg / pyunits.m**3,
          doc='Liquid density, kg/m^3')

m.dens_gas = Var(initialize=dens_gas,
          within=NonNegativeReals,
          units=pyunits.kg / pyunits.m**3,
          doc='Gas density, kg/m^3')

m.visc_liq = Var(initialize=visc_liq,
          bounds=(1E-5,1),
          units=pyunits.kg / (pyunits.m * pyunits.s),
          doc='Liquid viscocity, kg/m-s')

m.visc_gas = Var(initialize=visc_gas,
          bounds=(1E-9, 1),
          units=pyunits.kg / (pyunits.m * pyunits.s),
          doc='Gas viscocity, kg/m-s')

#m.Stl = Var(initialize=0.0742,
#              bounds=(0, 1),
#              units=pyunits.kg / pyunits.s**2,
#              doc='Critical surface tension of liquid (H2O), kg/s^2')

m.Stl_const1 = Var(initialize=235.8,
                   doc='Constant 1 to calculate critical surface tension of liquid (H2O)')

m.Stl_const2 = Var(initialize=647.096,
                   doc='Constant 2 (Critical temperature of pure water) to calculate critical surface tension of liquid (H2O)')

m.Stl_const3 = Var(initialize=1.256,
                   doc='Constant 3 to calculate critical surface tension of liquid (H2O)')

m.Stl_const4 = Var(initialize=0.625,
                   doc='Constant 4 to calculate critical surface tension of liquid (H2O)')

m.Ionic = Var(initialize=0.5,
              within=NonNegativeReals,
              units=pyunits.mol / pyunits.L,
              doc='Ionic strength based on NaCl, mol/L')

m.Salting_out_NaCl = Var(initialize=0.1,
                         within=NonNegativeReals,
                         units=pyunits.L / pyunits.mol,
                         doc='Setschenow, or salting-out, coefficient, L/mol')

m.logKOW = Var(initialize=2.49,
               within=NonNegativeReals,
               doc='Log Kow for pollutant')

m.Ks_const1 = Var(initialize=0.042,
                  within=NonNegativeReals,
                  doc='Setschenow, or salting-out, equation constant')

m.Ks_const2 = Var(initialize=0.113,
                  within=NonNegativeReals,
                  doc='Setschenow, or salting-out, equation constant')

m.activity = Var(initialize=1,
                 bounds=(1E-3, 1E1),
                 doc='Activity coefficient')

m.Enthalpies = Var(initialize=28.7,
                   within=NonNegativeReals,
                   units=pyunits.kJ / pyunits.mol,
                   doc='Standard enthalpies change of dissolution in water in kJ/mol at 25°C ')

m.H = Var(initialize=0.09,
          bounds=(1E-4, 1E1),
          doc='Default Henry"s constant without adjustments')

m.Hi = Var(initialize=0.1,
           bounds=(1E-2, 1E2),
           doc='Henry"s constant adjusted based on Ionic strength')

m.Ht = Var(initialize=0.450,
           bounds=(1E-3, 1E4),
           doc='Henry"s constant adjusted based on temperature')

m.Gas_to_liq_ratio_min = Var(initialize=2,
                             within=NonNegativeReals,
                             bounds=(1, 30),
                             doc='Minimum gas-to-liquid ratio')

m.Gas_to_liq_ratio = Var(initialize=QgQl,
                         within=NonNegativeReals,
                         bounds=(1, None),
                         doc='Optimum gas-to-liquid ratio')

m.Gas_to_liq_ratio_SF_const = Var(initialize=3.5,
                                  doc='Safety factor to obtain optimum gas-to-liquid ratio')

m.Strip_fact = Var(initialize=5,
                   bounds=(1, 20),
                   within=NonNegativeReals,
                   doc='Stripping factor')

m.P_drop = Var(initialize=Pdrop,
               bounds=(40, 1200),
               within=NonNegativeReals,
               doc='Pressure drop, Pa/m')

#m.Press_drop_F = Var(initialize=1.875,
#                     bounds=(1.5, 3.5),
#                     doc='Pressure drop calculation variable F') 

#m.Press_drop_a0 = Var(initialize=-2.28,
#                      bounds=(-3, 0),
#                      doc='Pressure drop calculation variable 0')

m.a0x = Var(initialize=-6.6599,
         doc='Pressure drop a0 equation variable x')

m.a0y = Var(initialize=4.3077,
         doc='Pressure drop a0 equation variable y')

m.a0z = Var(initialize=1.3503,
         doc='Pressure drop a0 equation variable z')

m.a0w = Var(initialize=0.15931,
         doc='Pressure drop a0 equation variable w')

#m.Press_drop_a1 = Var(initialize=-0.73,
#                      bounds=(-2, 0),
#                      doc='Pressure drop calculation variable 1')

m.a1x = Var(initialize=3.0945,
         doc='Pressure drop a1 equation variable x')

m.a1y = Var(initialize=4.3512,
         doc='Pressure drop a1 equation variable y')

m.a1z = Var(initialize=1.6240,
         doc='Pressure drop a1 equation variable z')

m.a1w = Var(initialize=0.20855,
         doc='Pressure drop a1 equation variable w')

#m.Press_drop_a2 = Var(initialize=-0.23,
#                      bounds=(-2, 0),
 #                     doc='Pressure drop calculation variable 2')

m.a2x = Var(initialize=1.7611,
         doc='Pressure drop a2 equation variable x')

m.a2y = Var(initialize=2.3394,
         doc='Pressure drop a2 equation variable y')

m.a2z = Var(initialize=0.89914,
         doc='Pressure drop a2 equation variable z')

m.a2w = Var(initialize=0.11597,
         doc='Pressure drop a2 equation variable w')

#m.Press_drop_E = Var(initialize=0.61, 
#                     bounds=(1E-1, 1E2),
#                     doc='Pressure drop calculation variable E') # CHECK-- --- scaling issues

#m.Press_drop_M = Var(initialize=1E-2,
#                     bounds=(1E-4, 0.45),
#                     doc='Pressure drop calculation variable M') # CHECK-- --- scaling issues

m.MassLoad_gas = Var(initialize=0.35,
                     bounds=(0.01, 2), # Bounds determined based on the literature
                     units=(pyunits.kg) / (pyunits.m**2 * pyunits.s),
                     doc='Gas mass loading rate, kg/m^2-s')

m.MassLoad_liq = Var(initialize=30,
                     bounds=(0.5, 45),
                     units=(pyunits.kg) / (pyunits.m**2 * pyunits.s),
                     doc='Liquid mass loading rate, kg/m^2-s') # CHECK-- --- --- range is affecting calculations

m.A_cross = Var(initialize=60,
                within=NonNegativeReals,
                bounds=(0, 1E3),
                units=pyunits.m**2,
                doc='Cross sectional area, m^2')

m.Diameter = Var(initialize=2.5,
                within=NonNegativeReals,
                bounds=(0.3, 8),
                units=pyunits.m,
                doc='Reactor cross diameter, m')

m.Diameter_pipes = Var(initialize=6,
                       within=NonNegativeReals,
                       bounds=(2, 24),
                       units=pyunits.inches,
                       doc='Pipe diameter, in')

m.Diameter_port = Var(initialize=6,
                      within=NonNegativeReals,
                      bounds=(2, 24),
                      units=pyunits.inches,
                      doc='Port diameter, in')

m.Critic_vol = Var(initialize=294,
                   within=NonNegativeReals,
                   units=pyunits.cm**3 / pyunits.mol,
                   doc='Critical volume of chemical, cm^3/mol')

#m.BoilingPoint_vol = Var(initialize=0.1,
#                         within=NonNegativeReals,
#                         units=pyunits.cm**3 / pyunits.mol,
#                         doc='Molar volume of solute at normal boiling point, cm^3/mol')

m.BoilingPoint_vol_const1 = Var(initialize=0.285,
                                doc='Constant 1 to calculate molar volume of solute at normal boiling point')

m.BoilingPoint_vol_const2 = Var(initialize=1.048,
                                doc='Constant 2 to calculate molar volume of solute at normal boiling point')

m.MWchem = Var(initialize=133.4,
               within=NonNegativeReals,
               units=pyunits.g / pyunits.mol,
               doc='MW of pollutant of focus, g/mol')

m.MWgas = Var(initialize=28.96,
              within=NonNegativeReals,
              units=pyunits.g / pyunits.mol,
              doc='MW of gas --- default for air = 28.96 g/mol')

#m.Diff_liq = Var(initialize=1E-10,
#                 bounds=(1E-24, 1E3),
#                 units=pyunits.m**2 / pyunits.s,
#                 doc='Liquid diffusion coefficient, m^2/s')

m.Diff_liq_const1 = Var(initialize=13.26E-9,
                        doc='Constant 1 in Hayduk_Laudie correlation')

m.Diff_liq_const2 = Var(initialize=1.14,
                        doc='Constant 2 in Hayduk_Laudie correlation')

m.Diff_liq_const3 = Var(initialize=0.589,
                        doc='Constant 3 in Hayduk_Laudie correlation')

#m.mol_sep_chem = Var(initialize=0.5,
#                   within=NonNegativeReals,
#                   units=pyunits.nm,
#                   doc='Molecular separation at collision variable for VOC, nm')

m.rA_const1 = Var(initialize=1.18,
                  doc='Molecular separation at collision equation constant')

#m.E_kA = Var(initialize=485,
#             within=NonNegativeReals,
#             doc='Molecular attraction of VOC')

m.E_kA_const1 = Var(initialize=1.21,
                  within=NonNegativeReals,
                  doc='Molecular attraction at collision equation constant')

#m.E_kA_avg = Var(initialize=485,
#                 within=NonNegativeReals,
#                 doc='Average molecular attraction of VOC')

m.T_norm_boiling = Var(initialize=347.1,
                 within=NonNegativeReals,
                 units=pyunits.K,
                 doc='Normal boiling point of chemical, K')

m.mol_sep_gas = Var(initialize=0.3711,
                   within=NonNegativeReals,
                   units=pyunits.nm,
                   doc='Molecular separation at collision variable for Air, nm')

m.E_kB = Var(initialize=78.6,
             within=NonNegativeReals,
             doc='Molecular attraction of Air')

#m.mol_sep_avg = Var(initialize=0.5,
#            within=NonNegativeReals,
 #           units=pyunits.nm,
 #           doc='Average molecular separation at collision variable for Air and VOC, nm')

#m.E_kAB = Var(initialize=100,
#              within=NonNegativeReals,
#              doc='Average molecular attraction')

#m.ee = Var(initialize=0.25,
#           doc='Constant to determine the collision function')

#m.E_cf = Var(initialize=0.25,
#             doc='Constant to determine the collision function')

m.E_cf_const1 = Var(initialize=-0.14329,
                  doc='Collision function equation constant')

m.E_cf_const2 = Var(initialize=0.48343,
                  doc='Collision function equation constant')

m.E_cf_const3 = Var(initialize=0.1939,
                  doc='Collision function equation constant')

m.E_cf_const4 = Var(initialize=0.13612,
                  doc='Collision function equation constant')

m.E_cf_const5 = Var(initialize=0.20578,
                  doc='Collision function equation constant')

m.E_cf_const6 = Var(initialize=0.083899,
                  doc='Collision function equation constant')

m.E_cf_const7 = Var(initialize=0.011491,
                  doc='Collision function equation constant')

#m.f = Var(initialize=0.1,
#          doc='Collision function variable')

#m.Diff_gas = Var(initialize=5E-6,
#           bounds=(1E-16, 1E1),
#           units=pyunits.m**2 / pyunits.s,
#           doc='Gas diffusion coefficient, m^2/s')

m.Diff_gas_const1 = Var(initialize=1.084,
                        within=NonNegativeReals,
                        doc='Gas diffusion coefficient eq constant')

m.Diff_gas_const2 = Var(initialize=0.249,
                        within=NonNegativeReals,
                        doc='Gas diffusion coefficient eq constant')

m.Load_liq = Var(initialize=1E-3,
                 within=NonNegativeReals,
                 doc='Liquid-phase velocity (volumetric loading rate), m3/m2-s')

m.Load_gas = Var(initialize=1E-1,
                 within=NonNegativeReals,
                 doc='Gas-phase velocity (volumetric loading rate), m3/m2-s')

#m.kl = Var(initialize=1E-4,
#           within=NonNegativeReals,
#           bounds=(1E-16, 1E2),
#           doc='Liquid-phase mass transfer rate coefficient, m/s')

#m.kg = Var(initialize=1E-4,
#           within=NonNegativeReals,
#           bounds=(1E-16, 1E2),
#           doc='Gas-phase mass transfer rate coefficient, m/s')

m.wet_area_pack = Var(initialize=250,
                      within=NonNegativeReals,
                      bounds=(0,2000),
                      doc='Wetted surface area of packing, m^2/m^3')

m.kla = Var(initialize=1E-2,
           within=NonNegativeReals,
           bounds=(1E-16, 1E4),
           doc='Liquid-phase mass transfer rate coefficient, 1/s')

m.HTU = Var(initialize=2.25,
            within=NonNegativeReals,
            bounds=(0,5), # Bounds determined based on the literature
            doc='Height for gas-phase mass transfer unit, m') 

m.NTU = Var(initialize=3.5,
            within=NonNegativeReals,
            bounds=(0,6), # Bounds determined based on the literature
            doc='Number of transfer units for gas-phase mass transfer unit')

m.Pack_height = Var(initialize=8,
                    bounds=(0, 30),
                    units=pyunits.m,
                    doc='Packing height in tower, m')

m.Tower_height = Var(initialize=10,
                     bounds=(0, 34),
                     units=pyunits.m,
                     doc='Tower height with a safety factor of 20"%, m')

m.Tower_heightSF = Var(initialize=1.2,
                     doc='Tower height safety factor of 20"%, m')

m.Qg = Var(initialize=0.5,
           within=NonNegativeReals,
           units=pyunits.m**3/pyunits.s,
           doc='Gas flowrate, m^3/s')

m.MassFlow_gas = Var(initialize=12,
                     within=NonNegativeReals,
                     units=pyunits.kg/pyunits.s,
                     doc='Gas mass flowrate, kg/s')

m.MassFlow_liq = Var(initialize=12,
                     within=NonNegativeReals,
                     units=pyunits.kg/pyunits.s,
                     doc='Liquid mass flowrate, kg/s')

m.P_amb = Var(initialize=101325,
           within=NonNegativeReals,
           units=pyunits.Pa,
           doc='Pressure in environment (assumed P = 1 atm), Pa')

m.P_loss = Var(initialize=20,
           within=NonNegativeReals,
           units=pyunits.Pa,
           doc='Pressure loss, Pa')

m.P_loss_const = Var(initialize=275,
                  doc='Empirical constant for pressure loss, N-s^2/m^4')

m.P_in = Var(initialize=105000,
             within=NonNegativeReals,
             units=pyunits.Pa,
             doc='Pressure at inlet, Pa')

m.Eff_blow = Var(initialize=0.40,
                   within=NonNegativeReals,
                   bounds=(0.05, 0.80),
                   doc='Blower efficiency')

m.Eff_pump = Var(initialize=0.85,
                   within=NonNegativeReals,
                   bounds=(0.05, 0.95),
                   doc='Pump efficiency')

m.Pow_blower = Var(initialize=5,
                   within=NonNegativeReals,
                   units=pyunits.kW,
                   doc='Blower brake power, kW')

m.Pow_pump = Var(initialize=5,
                 within=NonNegativeReals,
                 units=pyunits.kW,
                 doc='Liquid pump power, kW')

m.Total_Power = Var(initialize=10,
                    within=NonNegativeReals,
                    units=pyunits.kW,
                    doc='Total power requirement, kW')

m.SEC = Var(initialize=0.4,
            within=NonNegativeReals,
            units=pyunits.kWh/pyunits.m**3,
            doc='Specific energy demand, kWh/m^3')

m.cost_packing = Var(initialize=55000,
                     within=NonNegativeReals,
                     doc='Packing cost based on material, $/m^3')

m.cost_elec = Var(initialize=0.1,
                  within=NonNegativeReals,
                  doc='Electricity cost, $/kWh')

m.cost_scaling_2023_1 = Var(initialize=805.6/361.3,
                          within=NonNegativeReals,
                          doc='Scaling factor for year 2023 from year 1991')

m.cost_scaling_2023_2 = Var(initialize=805.6/532.9,
                          within=NonNegativeReals,
                          doc='Scaling factor for year 2023 from year 2010')

m.C1 = Var(initialize=100000,
           within=NonNegativeReals,
           doc='Process equipment cost for aluminum reactor (Dzombak et al., 2021; Dzombak et al., 1993), $')

m.C1_const1 = Var(initialize=45.2,
                   doc='Constant for C1')

m.C1_const2 = Var(initialize=3.5,
                   doc='Constant for C1')

m.C1_const3 = Var(initialize=0.0077,
                   doc='Constant for C1')

m.C2 = Var(initialize=100000,
           within=NonNegativeReals,
           doc='Access port cost (Dzombak et al., 2021; Dzombak et al., 1993), $')

m.C2_const1 = Var(initialize=-31.6,
                   doc='Constant for C2')

m.C2_const2 = Var(initialize=72.8,
                   doc='Constant for C2')

m.C2_const3 = Var(initialize=2.8,
                   doc='Constant for C2')

m.C2_const4 = Var(initialize=0.11,
                   doc='Constant for C2')

m.C3 = Var(initialize=100000,
           within=NonNegativeReals,
           doc='Inlet and outlet pipes cost (Dzombak et al., 2021; Dzombak et al., 1993), $')

m.C3_const1 = Var(initialize=133.8,
                   doc='Constant for C3')

m.C3_const2 = Var(initialize=42,
                   doc='Constant for C3')

m.C3_const3 = Var(initialize=4.8,
                   doc='Constant for C3')

m.C4 = Var(initialize=100000,
           within=NonNegativeReals,
           doc='Air inlet cost (Dzombak et al., 2021; Dzombak et al., 1993), $')

m.C4_const1 = Var(initialize=1.05,
                   doc='Constant for C4')

m.C5 = Var(initialize=100000,
           within=NonNegativeReals,
           doc='Tray rings cost (Dzombak et al., 2021; Dzombak et al., 1993), $')

m.C5_const1 = Var(initialize=70.4,
                  doc='Constant for C5')

m.C5_const2 = Var(initialize=4.45,
                  doc='Constant for C5')

m.C5_const3 = Var(initialize=1.73E-2,
                  doc='Constant for C5')

m.C6 = Var(initialize=100000,
           within=NonNegativeReals,
           doc='FRP packing support plate cost (Dzombak et al., 2021; Dzombak et al., 1993), $')

m.C6_const1 = Var(initialize=658.1,
                  doc='Constant for C6')

m.C6_const2 = Var(initialize=6.5,
                  doc='Constant for C6')

m.C6_const3 = Var(initialize=0.22,
                  doc='Constant for C6')

m.C7 = Var(initialize=100000,
           within=NonNegativeReals,
           doc='Plate-type liquid distributor cost (Dzombak et al., 2021; Dzombak et al., 1993), $')

m.C7_const1 = Var(initialize=20.6,
                  doc='Constant for C7')

m.C7_const2 = Var(initialize=1.1,
                  doc='Constant for C7')

m.C7_const3 = Var(initialize=9.7E-2,
                  doc='Constant for C7')

m.C8 = Var(initialize=100000,
           within=NonNegativeReals,
           doc='Mist eliminator cost (Dzombak et al., 2021; Dzombak et al., 1993), $')

m.C8_const1 = Var(initialize=46.4,
                  doc='Constant for C8')

m.C8_const2 = Var(initialize=9.3,
                  doc='Constant for C8')

m.C8_const3 = Var(initialize=0.14,
                  doc='Constant for C8')

m.C9 = Var(initialize=100000,
           within=NonNegativeReals,
           doc='Water pump capital cost (Smith D., 2005), $')

m.C9_const1 = Var(initialize=9.84E3,
                  doc='Constant for C9')

m.C9_const2 = Var(initialize=4,
                  doc='Constant for C9')

m.C9_const3 = Var(initialize=0.55,
                  doc='Constant for C9')

m.C10 = Var(initialize=100000,
           doc='Air blower capital cost (Towler & Sinnot, 2013), $')

m.C10_const1 = Var(initialize=4450,
                  doc='Constant for C10')

m.C10_const2 = Var(initialize=57,
                  doc='Constant for C10')

m.C10_const3 = Var(initialize=0.8,
                  doc='Constant for C10')

m.C11 = Var(initialize=135000,
           within=NonNegativeReals,
           doc='Packing material cost (Towler & Sinnot, 2013), $')

m.C12 = Var(initialize=100000,
           within=NonNegativeReals,
           doc='Administrative, laboratory, and building cost (Sharma, J.R., 2010), $')

m.CAPEX = Var(initialize=1000000,
           within=NonNegativeReals,
           doc='Total Capital Cost, 2023 $')

m.OC1 = Var(initialize=10000,
           within=NonNegativeReals,
           doc='Operational cost for water pump, $/yr')

m.OC2 = Var(initialize=10000,
           within=NonNegativeReals,
           doc='Operational cost for blower, $/yr')

m.OC3 = Var(initialize=10000,
           within=NonNegativeReals,
           doc='Operational cost for labor and maintenance, $/yr')

m.OC4 = Var(initialize=10000,
           within=NonNegativeReals,
           doc='Operational administrative, laboratory, and building cost (Sharma, J.R., 2010), $/yr')

m.OPEX = Var(initialize=1000000,
           within=NonNegativeReals,
           doc='Total Operational Cost, 2023 $/year')

m.lifetime_yrs = Var(initialize=25, 
                     bounds=(5, 100),
                     units=pyunits.year,
                     doc='System lifetime in years')

m.interest = Var(initialize=0.08, 
                 bounds=(0, 0.8),
                 doc='Purchase interest rate')

m.CCRF =  Var(initialize=1, 
             bounds=(0, 2E2),
              units=pyunits.year,
              doc='Capital Cost Recovery Factor (CCRF)')

m.LCOW = Var(initialize=0.1, 
             bounds=(0, 1E4),
              units=pyunits.dimensionless,
              doc='Total Levelized Water Cost in Treatment System, $/m3')     


# Equation and Constraints
#--------------------------------------------------------------------------------------------------------------------------
set_scaling_factor(m.Ce, 1E4)
#set_scaling_factor(m.Press_drop_M, 1E6)
#set_scaling_factor(m.Diff_liq, 1E10)
#set_scaling_factor(m.Diff_gas, 1E6)
set_scaling_factor(m.P_amb, 1E-5)
set_scaling_factor(m.P_in, 1E-5)
#set_scaling_factor(m.Hi, 1E2)
#set_scaling_factor(m.Ht, 1E2)
#set_scaling_factor(m.activity, 1E1)
#set_scaling_factor(m.Gas_to_liq_ratio_min, 1E2)
#set_scaling_factor(m.Gas_to_liq_ratio, 1E-1)
#set_scaling_factor(m.wet_area_pack, 1E-2)
#set_scaling_factor(m.Press_drop_a0, 1E1)
#set_scaling_factor(m.Press_drop_a1, 1E1)
#set_scaling_factor(m.Press_drop_a2, 1E1)
#set_scaling_factor(m.Press_drop_E, 1E1)
#set_scaling_factor(m.Press_drop_M, 1E3)
#set_scaling_factor(m.MassLoad_gas, 1E2)
#set_scaling_factor(m.MassLoad_liq, 1E-2)
#set_scaling_factor(m.Load_liq, 1E4)
#set_scaling_factor(m.Load_gas, 1E2)
#set_scaling_factor(m.kl, 1E4)
#set_scaling_factor(m.kg, 1E3)
#set_scaling_factor(m.kla, 1E2)


# Expre = 19 orig_contraint = 155

#m.Stl_constr = Constraint(expr=m.Stl == (235.8 * ((1 - (m.Tl_K/647.096))**1.256) * (1 - 0.625*(1 - (m.Tl_K/647.096))))/1000)
m.Stl = Expression(expr=(m.Stl_const1 * ((1 - (m.Tl_K/m.Stl_const2))**m.Stl_const3) * (1 - m.Stl_const4*(1 - (m.Tl_K/m.Stl_const2))))/1000)     # 1,000 is a conversion value to convert from mN/m to N/m (kg/s^2) Ref: http://www.iapws.org/relguide/Surf-H2O-2014.pdf

if Henrys_const == "calculated":
    m.Ionic.fix(Ionic_strength)
    m.logKOW.fix(kow)
    m.Enthalpies.fix(enth)
    m.H.fix(H)
    m.Salting_out_NaCl_constr = Constraint(expr=m.Salting_out_NaCl == (m.Ks_const1 * m.logKOW) + m.Ks_const2) 
    #m.Salting_out_NaCl = Expression(expr=(m.Ks_const1 * m.logKOW) + m.Ks_const2) 
    
    m.activity_constr = Constraint(expr=m.activity == exp(m.Salting_out_NaCl * m.Ionic)) 
    #m.activity = Expression(expr=exp(m.Ks_NaCl * m.Ionic)) 

    m.Hi_constr = Constraint(expr=m.Hi == m.activity * m.H) 
    #m.Hi = Expression(expr=m.activity * m.H) 

    m.Ht_constr = Constraint(expr=m.Ht == m.Hi * exp((-m.Enthalpies/m.R)*((1/m.Tl_K)-(1/298.15)))) # The 298.15 is the reference temperature of the default Henry's constant at 25 C
    #m.Ht = Expression(expr=m.Hi * exp((-m.Enthalpies/m.R)*((1/m.Tl_K)-(1/298.15)))) 
else:
    m.Ht.fix(H)

if air_to_water_ratio == "calculated":
    m.Ce.fix(Ce)

    m.QgQl_min_constr = Constraint(expr=m.Gas_to_liq_ratio_min == (m.Ci - m.Ce)/(m.Ci*m.Ht))

    m.QgQl_constr = Constraint(expr=m.Gas_to_liq_ratio == m.Gas_to_liq_ratio_SF_const * m.Gas_to_liq_ratio_min) 
else:
    m.Gas_to_liq_ratio.fix(QgQl)
    
    m.Ce_constr = Constraint(expr=m.Ce == m.Ci - (m.Gas_to_liq_ratio/3.5)*(m.Ci*m.Ht)) 
    #m.Ce = Expression(expr=m.Ci - (m.Gas_to_liq_ratio/3.5)*(m.Ci*m.Ht))

m.Qg_constr = Constraint(expr=m.Qg == m.Gas_to_liq_ratio * m.Ql)

m.Strip_fact_constr = Constraint(expr=m.Strip_fact == m.Gas_to_liq_ratio * m.Ht) 
#m.Strip_fact = Expression(expr=m.Gas_to_liq_ratio * m.Ht) 

#m.Press_drop_F_constr = Constraint(expr=m.Press_drop_F == log10(m.P_drop))
m.Press_drop_F = Expression(expr=log10(m.P_drop))

#m.Press_drop_a0_constr = Constraint(expr=m.Press_drop_a0 == m.a0x + (m.a0y*m.Press_drop_F) - (m.a0z*(m.Press_drop_F**2)) + (m.a0w*(m.Press_drop_F**3)))
m.Press_drop_a0= Expression(expr=m.a0x + (m.a0y*m.Press_drop_F) - (m.a0z*(m.Press_drop_F**2)) + (m.a0w*(m.Press_drop_F**3)))

#m.Press_drop_a1_constr = Constraint(expr=m.Press_drop_a1 == m.a1x - (m.a1y*m.Press_drop_F) + (m.a1z*(m.Press_drop_F**2)) - (m.a1w*(m.Press_drop_F**3)))
m.Press_drop_a1 = Expression(expr=m.a1x - (m.a1y*m.Press_drop_F) + (m.a1z*(m.Press_drop_F**2)) - (m.a1w*(m.Press_drop_F**3)))

#m.Press_drop_a2_constr = Constraint(expr=m.Press_drop_a2 == m.a2x - (m.a2y*m.Press_drop_F) + (m.a2z*(m.Press_drop_F**2)) - (m.a2w*(m.Press_drop_F**3)))
m.Press_drop_a2 = Expression(expr=m.a2x - (m.a2y*m.Press_drop_F) + (m.a2z*(m.Press_drop_F**2)) - (m.a2w*(m.Press_drop_F**3)))

#m.Press_drop_E_constr = Constraint(expr=m.Press_drop_E == -log10((m.Gas_to_liq_ratio)*sqrt((m.dens_gas/m.dens_liq)-(m.dens_gas/m.dens_liq)**2)))
m.Press_drop_E = Expression(expr=-log10((m.Gas_to_liq_ratio)*sqrt((m.dens_gas/m.dens_liq)-(m.dens_gas/m.dens_liq)**2)))

#m.Press_drop_M_constr = Constraint(expr=m.Press_drop_M == 10**(m.Press_drop_a0 + (m.Press_drop_a1*m.Press_drop_E) + (m.Press_drop_a2*(m.Press_drop_E**2))))
m.Press_drop_M = Expression(expr=10**(m.Press_drop_a0 +  (m.Press_drop_a1*m.Press_drop_E) + (m.Press_drop_a2*(m.Press_drop_E**2))))                   
#m.Press_drop_M.fix(0.0015)

m.MassLoad_gas_constr = Constraint(expr=m.MassLoad_gas == sqrt((m.Press_drop_M * m.dens_gas * (m.dens_liq-m.dens_gas))/((m.pack_fact)*(m.visc_liq**0.1))))
#m.MassLoad_gas = Expression(expr=sqrt((m.Press_drop_M * m.dens_gas * (m.dens_liq-m.dens_gas))/((m.pack_fact)*(m.visc_liq**0.1))))                   

m.MassLoad_liq_constr = Constraint(expr=m.MassLoad_liq == m.MassLoad_gas/((m.Gas_to_liq_ratio)*(m.dens_gas/m.dens_liq)))
#m.MassLoad_liq = Expression(expr=m.MassLoad_gas/((m.Gas_to_liq_ratio)*(m.dens_gas/m.dens_liq)))           

m.MassFlow_gas_constr = Constraint(expr=m.MassFlow_gas == m.Qg * m.dens_gas)

m.MassFlow_liq_constr = Constraint(expr=m.MassFlow_liq == m.Ql * m.dens_liq)

m.A_cross_constr = Constraint(expr=m.A_cross == m.MassFlow_liq/m.MassLoad_liq)

m.Diameter_constr = Constraint(expr=m.Diameter == sqrt((4*m.A_cross)/pi))

m.Load_liq_constr = Constraint(expr=m.Load_liq == m.Ql/m.A_cross) #---------------------------------------------------------------------- constraint scaling issue, when scaled the calculation output is incorrect
#m.Load_liq = Expression(expr= m.Ql/m.A_cross)

m.Load_gas_constr = Constraint(expr=m.Load_gas == m.Qg/m.A_cross)
#m.Load_gas = Expression(expr= m.Qg/m.A_cross)

if mass_transfer_coeff_kla == "calculated":
    #m.BoilingPoint_vol_constr = Constraint(expr=m.BoilingPoint_vol == 0.285 * m.Critic_vol**1.048) 
    m.BoilingPoint_vol = Expression(expr=m.BoilingPoint_vol_const1 * m.Critic_vol**m.BoilingPoint_vol_const2)           

    #m.Diff_liq_constr = Constraint(expr=m.Diff_liq == m.Diff_liq_const1/(((m.visc_liq/0.001)**1.14)*(m.BoilingPoint_vol **0.589)))  
    m.Diff_liq = Expression(expr=m.Diff_liq_const1/(((m.visc_liq/0.001)**m.Diff_liq_const2)*(m.BoilingPoint_vol**m.Diff_liq_const3))) #   The 0.001 is a conversion from kg/m-s to cP   

    #m.mol_sep_chem_constr = Constraint(expr=m.mol_sep_chem == m.rA_const1 * ((m.BoilingPoint_vol/1000)**(1/3))) # Vb to L/mol
    m.mol_sep_chem = Expression(expr=m.rA_const1 * ((m.BoilingPoint_vol/1000)**(1/3))) # the 1000 is a conversion value from cm3/mol to L/mol

    #m.E_kA_constr = Constraint(expr=m.E_kA == m.E_kA_const1 * m.Tboiling) 
    m.E_kA  = Expression(expr=m.E_kA_const1 * m.T_norm_boiling)           

    #m.mol_sep_avg_constr = Constraint(expr=m.mol_sep_avg == 0.5 * (m.mol_sep_chem+m.mol_sep_gas))
    m.mol_sep_avg  = Expression(expr=0.5 * (m.mol_sep_chem+m.mol_sep_gas))           

    #m.E_kAB_constr = Constraint(expr=m.E_kAB == sqrt(m.E_kA * m.E_kB))
    m.E_kAB  = Expression(expr=sqrt(m.E_kA * m.E_kB))           

    #m.ee_constr = Constraint(expr=m.ee == log10(m.Tl_K / m.E_kAB))
    m.ee  = Expression(expr=log10(m.Tl_K / m.E_kAB))           

    #m.E_cf_constr = Constraint(expr=m.E_cf ==
    #                           m.E_cf_const1-
    #                           m.E_cf_const2*(m.ee)+
    #                           m.E_cf_const3*(m.ee**2)+
    #                           m.E_cf_const4*(m.ee**3)-
    #                           m.E_cf_const5*(m.ee**4)+
    #                           m.E_cf_const6*(m.ee**5)-
    #                           m.E_cf_const7*(m.ee**6))
    m.E_cf = Expression(expr=m.E_cf_const1 -
                        m.E_cf_const2*(m.ee)+
                        m.E_cf_const3*(m.ee**2)+
                        m.E_cf_const4*(m.ee**3)-
                        m.E_cf_const5*(m.ee**4)+
                        m.E_cf_const6*(m.ee**5)-
                        m.E_cf_const7*(m.ee**6))

    #m.f_constr = Constraint(expr=m.f == 10**m.E_cf)
    m.f  = Expression(expr=10**m.E_cf)           

    #m.Diff_gas_constr = Constraint(expr=m.Diff_gas == 0.0001 *
    #            (((m.Diff_gas_const1-(m.Diff_gas_const2*sqrt((1/m.MWchem)+(1/m.MWgas))))*(m.Tl_K**1.5)*(sqrt((1/m.MWchem)+(1/m.MWgas))))
    #            /
    #            ((m.P_amb)*(m.mol_sep_avg**2)*(m.f)))) 
    m.Diff_gas = Expression(expr=0.0001 * (((m.Diff_gas_const1-(m.Diff_gas_const2*sqrt((1/m.MWchem)+(1/m.MWgas))))*(m.Tl_K**1.5)*(sqrt((1/m.MWchem)+(1/m.MWgas)))) # the 0.0001 is used to convert cm2/s to m2/s
                                        /
                                        ((m.P_amb)*(m.mol_sep_avg**2)*(m.f)))) 

    m.wet_area_pack_constr = Constraint(expr=m.wet_area_pack ==m.pack_surface_area * (1 - exp(-1.45*
                                                            ((m.Stc / m.Stl)**0.75)*
                                                            ((m.MassLoad_liq / (m.pack_surface_area * m.visc_liq))**0.1)*
                                                            ((((m.MassLoad_liq**2)*m.pack_surface_area) / ((m.dens_liq**2) * m.g))**-0.05)*
                                                            (((m.MassLoad_liq**2) / (m.dens_liq * m.pack_surface_area * m.Stl))**0.2)))
                                                            )

    #m.kl_constr = Constraint(expr=m.kl == 0.0051 *
    #                        (((m.MassLoad_liq / (m.wet_area_pack * m.visc_liq)))**(2/3))*
    #                        ((m.visc_liq / (m.dens_liq * m.Diff_liq))**(-1/2))*
    #                        ((m.pack_surface_area * m.pack_diam)**0.4)*
    #                        ((m.dens_liq / (m.visc_liq* m.g))**(-1/3))
    #                        )
    m.kl = Expression(expr=0.0051 *
                            (((m.MassLoad_liq / (m.wet_area_pack * m.visc_liq)))**(2/3))*
                            ((m.visc_liq / (m.dens_liq * m.Diff_liq))**(-1/2))*
                            ((m.pack_surface_area * m.pack_diam)**0.4)*
                            ((m.dens_liq / (m.visc_liq* m.g))**(-1/3))
                            )

    #m.kg_constr = Constraint(expr=m.kg == 5.23*
    #                        (m.pack_surface_area*m.Diff_gas)*
    #                        ((m.MassLoad_gas / (m.pack_surface_area * m.visc_gas))**0.7)*
    #                        ((m.visc_gas / (m.dens_gas * m.Diff_gas))**(1/3))*
    #                        ((m.pack_surface_area * m.pack_diam)**-2)
    #                        )
    m.kg = Expression(expr=5.23*
                            (m.pack_surface_area*m.Diff_gas)*
                            ((m.MassLoad_gas / (m.pack_surface_area * m.visc_gas))**0.7)*
                            ((m.visc_gas / (m.dens_gas * m.Diff_gas))**(1/3))*
                            ((m.pack_surface_area * m.pack_diam)**-2)
                            ) 

    m.kla_constr  = Constraint(expr=m.kla == 0.7*(1/(((1/(m.kl*m.wet_area_pack)) + (1/(m.kg*m.wet_area_pack*m.Ht)))))) # the 0.7 is a commonly used safety factor for AS units mass transf predictions
    #m.kla  = Expression(expr=1/(0.75*((1/(m.kl*m.wet_area_pack)) + (1/(m.kg*m.wet_area_pack*m.Ht)))))

else:
    m.kla.fix(kLa)

m.HTU_constr = Constraint(expr=m.HTU == m.Load_liq/m.kla) 

m.NTU_constr = Constraint(expr=m.NTU == (m.Strip_fact/(m.Strip_fact-1))*log((1+(m.Ci/m.Ce)*(m.Strip_fact-1))/m.Strip_fact)) 

m.Pack_height__constr = Constraint(expr=m.Pack_height == m.HTU * m.NTU)

m.Tower_height_constr = Constraint(expr=m.Tower_height == m.Pack_height * m.Tower_heightSF) 

m.P_loss_constr = Constraint(expr=m.P_loss == (m.Load_gas**2) * m.P_loss_const)

m.P_in_constr = Constraint(expr=m.P_in == m.P_amb + (m.P_drop * m.Tower_height) + m.P_loss)

m.Pow_blower_constr = Constraint(expr=m.Pow_blower == 
                                 ((m.MassFlow_gas * (m.R * 1000) * m.Tg_K) / (m.MWgas * 0.283 * m.Eff_blow)) *
                                 (((m.P_in/m.P_amb)**0.283) - 1)) # 1,000 used to conver kJ/mol to J/mol

m.Pow_pump_constr = Constraint(expr=m.Pow_pump == ((m.MassFlow_liq * m.Tower_height * m.g) / m.Eff_pump) * (1/1000)) # 1/1000 Conversion from Watts to kW


if pack_type == 'plastic':
    m.cost_packing.fix(5500)

elif pack_type == 'ceramic':
    m.cost_packing.fix(2000)

elif pack_type == 'steel':
    m.cost_packing.fix(8000)


m.C1_constr = Constraint(expr=m.C1 == ((m.C1_const1 + 
                         (m.C1_const2*(m.Diameter * 39.3701)) - 
                         (m.C1_const3*((m.Diameter * 39.3701)**2))) * m.Tower_height) * m.cost_scaling_2023_1) # the 39.307 aims at converting meters to inches

m.C2_constr = Constraint(expr=m.C2 == (m.C2_const1 + 
                         (m.C2_const2*m.Diameter_port) - 
                         (m.C2_const3*(m.Diameter_port)**2) +
                         (m.C2_const4*(m.Diameter_port)**3)) * (m.cost_scaling_2023_1))

m.C3_constr = Constraint(expr=m.C3 == (m.C3_const1 + 
                         (m.C3_const2*m.Diameter_pipes) - 
                         (m.C3_const3*(m.Diameter_pipes)**2)) * (m.cost_scaling_2023_1))

m.C4_constr = Constraint(expr=m.C4 == m.C4_const1 * m.C2)

m.C5_constr = Constraint(expr=m.C5 == (m.C5_const1 + 
                                       (m.C5_const2*m.Diameter)+
                                       (m.C5_const3*(m.Diameter**2))) * (m.cost_scaling_2023_1))

m.C6_constr = Constraint(expr=m.C6 == (m.C6_const1 + 
                                       (m.C6_const2*m.Diameter) +
                                       (m.C6_const3*(m.Diameter**2))) * (m.cost_scaling_2023_1))

m.C7_constr = Constraint(expr=m.C7 == (m.C7_const1 + 
                                       (m.C7_const2*m.Diameter) +
                                       (m.C7_const3*(m.Diameter**2))) * (m.cost_scaling_2023_1))

m.C8_constr = Constraint(expr=m.C8 == (m.C8_const1 + 
                                       (m.C8_const2*m.Diameter)+
                                       (m.C8_const3*(m.Diameter**2))) * (m.cost_scaling_2023_1))

m.C9_constr = Constraint(expr=m.C9 == (m.C9_const1 * (m.Pow_pump/m.C9_const2)**m.C9_const3) * (m.cost_scaling_2023_1))

m.C10_constr = Constraint(expr=m.C10 == (m.C10_const1 + m.C10_const2*(m.Qg * 3600)**m.C10_const3) * (m.cost_scaling_2023_1)) # 3600 to convert m3/s to m3/hr

m.C11_constr = Constraint(expr=m.C11 == ((m.Pack_height * m.A_cross) * m.cost_packing) * (m.cost_scaling_2023_2))

m.C12_constr = Constraint(expr=m.C12 == (69195*(m.Ql*22.82)**0.5523) * (m.cost_scaling_2023_2)) # the 22.82 is to convert from m3/s to MGD

m.OC1_constr = Constraint(expr=m.OC1 == m.Pow_pump * 8760 * m.cost_elec) # Cost for electricity assumed as $0.1/kWh

m.OC2_constr = Constraint(expr=m.OC2 == m.Pow_blower * 8760 * m.cost_elec) # Cost for electricity assumed as $0.1/kWh

m.OC3_constr = Constraint(expr=m.OC3 ==m.CAPEX * 0.01) # Assumed that 1% of the total CAPEX is O&M

m.OC6_constr = Constraint(expr=m.OC4 == (88589*(m.Ql*22.82)**0.4529) * (m.cost_scaling_2023_2)) # the 22.82 is to convert from m3/s to MGD

m.Total_Power_constr = Constraint(expr=m.Total_Power == m.Pow_blower + m.Pow_pump)

m.SEC_constr = Constraint(expr=m.SEC == m.Total_Power / (m.Ql * 3600)) # 3600 to conver m3/s to m3/hr

m.CCRF_constr = Constraint(expr=m.CCRF == (((1+m.interest)**m.lifetime_yrs)-1)/(m.interest*(m.interest+1)**m.lifetime_yrs))

m.CAPEX_constr = Constraint(expr=m.CAPEX == m.C1+m.C2+m.C3+m.C4+m.C5+m.C6+m.C7+m.C8+m.C9+m.C10+m.C11+m.C12)

m.OPEX_constr = Constraint(expr=m.OPEX == m.OC1+m.OC2+m.OC3+m.OC4)

m.LCOW_constr = Constraint(expr=m.LCOW == ((m.CAPEX/m.CCRF)+m.OPEX)/(m.Ql*86400*365)) # conversion from m3/s to m3/year

# Constraint deactivation
#m.Press_drop_M_constr.deactivate()
#m.MassLoad_gas_constr.deactivate()
#m.MassLoad_liq_constr.deactivate()

#constraint_scaling
constraint_scaling_transform(m.Load_liq_constr,1E-12)
#constraint_scaling_transform(m.Ce_constr,1E-12)
#constraint_scaling_transform(m.activity_constr,1E2)
#constraint_scaling_transform(m.Hi_constr,1E2)
#constraint_scaling_transform(m.Ht_constr,1E-16)
#constraint_scaling_transform(m.Press_drop_M_constr,1E-10)
#constraint_scaling_transform(m.kl_constr,1E-20)
#constraint_scaling_transform(m.kg_constr,1E-20)
#constraint_scaling_transform(m.kla_constr,1E-16)
#constraint_scaling_transform(m.Diff_gas_constr,1E-6)
#constraint_scaling_transform(m.Diff_liq_constr,1E-10)

#m.constraint_transformed_scaling_factor.display()

# Fixed variables
#--------------------------------------------------------------------------------------------------------------------------
m.P_drop.fix()
m.Ci.fix()
#m.Ce.fix()
m.H.fix()
m.Tl_K.fix()
m.Tg_K.fix()
m.T_norm_boiling.fix()
m.logKOW.fix()
m.R.fix()
m.Enthalpies.fix()
m.g.fix()
m.dens_liq.fix()
m.dens_gas.fix()
m.Stl_const1.fix()
m.Stl_const2.fix()
m.Stl_const3.fix()
m.Stl_const4.fix()
m.Critic_vol.fix()
m.BoilingPoint_vol_const1.fix()
m.BoilingPoint_vol_const2.fix()
m.Gas_to_liq_ratio_SF_const.fix()
m.visc_liq.fix()
m.visc_gas.fix()
m.cost_elec.fix()
m.Ks_const1.fix()
m.Ks_const2.fix()
m.a0x.fix()
m.a0y.fix()
m.a0z.fix()
m.a0w.fix()
m.a1x.fix()
m.a1y.fix()
m.a1z.fix()
m.a1w.fix()
m.a2x.fix()
m.a2y.fix()
m.a2z.fix()
m.a2w.fix()
m.Diff_liq_const1.fix()
m.Diff_liq_const2.fix()
m.Diff_liq_const3.fix()
m.P_amb.fix()
m.rA_const1.fix()
m.E_kA_const1.fix()
m.mol_sep_gas.fix()
m.E_kB.fix()
m.E_cf_const1.fix()
m.E_cf_const2.fix()
m.E_cf_const3.fix()
m.E_cf_const4.fix()
m.E_cf_const5.fix()
m.E_cf_const6.fix()
m.E_cf_const7.fix()
m.Diff_gas_const1.fix()
m.Diff_gas_const2.fix()
m.MWgas.fix()
m.MWchem.fix()
m.Tower_heightSF.fix()
m.P_loss_const.fix()
m.Eff_blow.fix()
m.Eff_pump.fix()
m.cost_elec.fix()
m.cost_scaling_2023_1.fix()
m.cost_scaling_2023_2.fix()
m.C1_const1.fix()
m.C1_const2.fix()
m.C1_const3.fix()
m.C2_const1.fix()
m.C2_const2.fix()
m.C2_const3.fix()
m.C2_const4.fix()
m.C3_const1.fix()
m.C3_const2.fix()
m.C3_const3.fix()
m.C4_const1.fix()
m.C5_const1.fix()
m.C5_const2.fix()
m.C5_const3.fix()
m.C6_const1.fix()
m.C6_const2.fix()
m.C6_const3.fix()
m.C7_const1.fix()
m.C7_const2.fix()
m.C7_const3.fix()
m.C8_const1.fix()
m.C8_const2.fix()
m.C8_const3.fix()
m.C9_const1.fix()
m.C9_const2.fix()
m.C9_const3.fix()
m.C10_const1.fix()
m.C10_const2.fix()
m.C10_const3.fix()
m.Diameter_pipes.fix()
m.Diameter_port.fix()
m.lifetime_yrs.fix()
m.interest.fix()

#--------------------------------------------------------------------------------------------------------------------------
calculate_scaling_factors(m)
dof = degrees_of_freedom(m)
print(f'\nDegrees of Freedom: {dof}\n')

solver = get_solver("ipopt-watertap")

results = solver.solve(m)
print(f'Model solve = {results.solver.termination_condition}\n')

print('Number of variables in model: ',number_variables(m))
print('Number of constraints in model: ', number_total_constraints(m), end='\n\n')
print('Infeasible_constraints:')
print(print_infeasible_constraints(m), end='\n\n')
print('Infeasible_bounds:')
print(print_infeasible_bounds(m), end='\n\n')
print('Variables_close_to_bounds:')
print(print_variables_close_to_bounds(m), end='\n\n')
print('Constraints_close_to_bounds:')
print(print_constraints_close_to_bounds(m), end='\n\n')
m.display()

In [ ]:
print('------------------------------------------------------------------------------------------------------------------------------------------')
print('User inputs:')
print('Liquid Flowrate: ', m.Ql(),'m^3/s')
print('Initial concentration of pollutant: ', m.Ci(),'mg/L')
if air_to_water_ratio == "calculated":
    print('Target concentration of pollutant specified by user: ', m.Ce(),'mg/L')
else:
    print('Optimal air to water ratio: %3f'%m.Gas_to_liq_ratio())
print('Influent ionic strength: ', m.Ionic(),'M')
print('Liquid temperature: ', m.Tl(),'°C')
print('Gas temperature: ', m.Tg(),'°C')
print('Liquid density: ', m.dens_liq(),'kg/m^3')
print('Gas density: ', m.dens_gas(),'kg/m^3')
print('Liquid viscocity: ', m.visc_liq(),'kg/m-s')
print('Gas viscocity: ', m.visc_gas(),'kg/m-s')
print(end='\n\n')
print('Packing inputs:')
print('Specific surface area of packing: ', m.pack_surface_area(),'m^2/m^3')
print('Nominal packing diameter: ', m.pack_diam(),'m')
print('Packing factor: ', m.pack_fact(),'m^2/m^3')
print('Critical surface tension of packing: ', m.Stc(),'kg/s^2')
print(end='\n\n')
"""
print('Pollutant default properties:')
print('Henrys Dimensionless Constant at 25°C: ', m.H())
print('Molecular weight: ', m.MWgas(),'g/mol')
print('Critical volume: ', m.Critic_vol(),'cm^3/mol')
print('Standard enthalpies change of dissolution in water at 25°C: ', m.Enthalpies(),'kJ/mol')
print('The Octanol/Water partition coefficient: ', m.logKOW())
print('Boiling temperature: ', m.T_norm_boiling(),'K')
print(end='\n\n')
"""
print('System Design Results:')
print('Critical surface tension of liquid (H2O)', m.Stl(),'kg/s^2')
if Henrys_const == "calculated":
    print('Salting-out coefficient for NaCl and pollutant: %.3f'%m.Salting_out_NaCl())
    print('Activity of salinity: %3f'%m.activity())
    print('Henrys constant adjusted for salinity: %.3f'%m.Hi())
    print('Henrys constant adjusted for temperature: %.3f'%m.Ht())
if air_to_water_ratio == "calculated":
    print('Minimum air to water ratio: %.3f'%m.Gas_to_liq_ratio_min())
    print('Optimal air to water ratio: %3f'%m.Gas_to_liq_ratio())
else:
    print('Effluent concentration of pollutant: ', m.Ce(),'mg/L')
print('Gas flow: %0f'%m.Qg(),'m^3/s')
print('Stripping factor of pollutant: %.2f'%m.Strip_fact())
#print('Pressure drop variable E to determine pressure drop variable M: ', m.Press_drop_E())
print('Pressure drop variable M to determine gas mass loading rate: ', m.Press_drop_M())
print('Gas mass loading rate: %.3f'%m.MassLoad_gas(),'kg/m^2-s')
print('Liquid mass loading rate: %.3f'%m.MassLoad_liq(),'kg/m^2-s')
print('Cross sectional area of column: %.3f'%m.A_cross(),'m^2')
print('Diameter of column: %.3f'%m.Diameter(),'m')
print('Liquid superficial velocity: %.5f'%m.Load_liq(),'m^3/m^2-s')
print('Gas superficial velocity: %.4f'%m.Load_gas(),'m^3/m^2-s')

if mass_transfer_coeff_kla == "calculated":
    print('Molar volume at boiling point of pollutant: %.3f'%m.BoilingPoint_vol(),'cm^3/mol')
    #print(m.mol_sep_chem()) 
    #print(m.E_kA())        
    #print(m.mol_sep_avg())         
    #print(m.E_kAB())       
    #print(m.ee())        
    #print(m.E_cf())
    #print(m.f())
    print('Liquid diffusion coefficient: ',m.Diff_liq(),'m^2/s')
    print('Gas diffusion coefficient: ',m.Diff_gas(),'m^2/s')
    print(end='\n')
    print('OTO mass transfer model results:')
    print('Liquid phase mass transfer coefficient:', m.kl(),'m/s')
    print('Gas phase mass transfer coefficient:', m.kg(),'m/s')
    print('Wetted surface area of packing: %.1f' %m.wet_area_pack(),'m^2/m^3')

print('Overall liquid phase mass transfer coefficient:', m.kla(),'1/s')
print('Overall Height of Transfer Units (HTU): %.3f'%m.HTU(),'m')
print('Overall Number of Transfer Units (NTU): %.3f'%m.NTU())
print('Packing height: %.3f'%m.Pack_height(),'m')
print('Total tower height: %.3f'%m.Tower_height(),'m')
print(end='\n\n')
print('Energy Results:')
print('Pressure loss in tower: ', m.P_loss(), 'Pa')
print('Pressure input in tower: ', m.P_in(), 'Pa')
print('Blower brake power: ', m.Pow_blower(), 'kW')
print('Pumping brake power: ', m.Pow_pump(), 'kW')
print('Total power: ', m.Total_Power(), 'kW')
print('Specific energy consumption: ', m.SEC(), 'kWh/m^3')
print(end='\n\n')
print('Cost Results:')
print('Process equipment cost for aluminum material tower: $ %.2f'%m.C1())
print('Access port cost: $ %.2f'%m.C2())
print('Inlet and outlet pipe cost: $ %.2f'%m.C3())
print('Air inlet cost: $ %.2f'%m.C4())
print('Tray ring cost: $ %.2f'%m.C5())
print('FRP packing support plate cost: $ %.2f'%m.C6())
print('Plate-type liquid distributor cost: $ %.2f'%m.C7())
print('Mist eliminator cost: $ %.2f'%m.C8())
print('Water pump cost: $ %.2f'%m.C9())
print('Air blower cost: $ %.2f'%m.C10())
print('Packing material cost: $ %.2f'%m.C11())
print('Administrative, laboratory, and building cost: $ %.2f'%m.C12())
print('Total Capital Cost: $ %.2f'%m.CAPEX())
print(end='\n')
print('Water pump energy cost ($0.1/kWh): $ %.2f'%m.OC1(),'/yr')
print('Air blower energy cost ($0.1/kWh): $ %.2f'%m.OC2(),'/yr')
print('Labor and maintenance cost: $ %.2f'%m.OC3(),'/yr')
print('Administrative, laboratory, and building cost: $ %.2f'%m.OC4(),'/yr')
print('Total Operational Cost: $ %.2f'%m.OPEX(),'/yr')
print(end='\n')
print('Levelized Cost of Water: $ %.2f'%m.LCOW(),'/m^3')